In [1]:
import numpy as np
import tensorflow as tf

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
sample = ' my name is Ethan' #한칸을 띄면 my name is로, 안띄면 y name is 로..
idx2char = list(set(sample)) #idx2char
char2idx = {c:i for i,c in enumerate(idx2char)} #역으로 바꾸기

In [3]:
batch_size = 1 #통째로 한문장을 학습하겠다
input_dim = len(idx2char)
hidden_size = len(idx2char)
num_classes = len(idx2char)
sequence_length = len(sample)-1 #끝에 한글자를 뺸다 hihell 생각하면됨, 여러 문장일 시 내맘대로 정하기
learning_rate= 0.1

sample_idx =  [char2idx[c] for c in sample]
x_data = [sample_idx[:-1]]
y_data =[sample_idx[1:]]


In [4]:
X = tf.placeholder(tf.int32, shape = [None, sequence_length, input_dim])
Y = tf.placeholder(tf.int32, shape = [None, sequence_length])

x_one_hot = tf.one_hot(X,input_dim)
# x_one_hot = tf.reshape(x_one_hot,shape=[-1,input_dim])

cell = tf.contrib.rnn.BasicLSTMCell(num_units = hidden_size, state_is_tuple=True) 
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, X, initial_state=initial_state, dtype=tf.float32)

X_for_fc = tf.reshape(outputs, [-1,hidden_size]) #flatten 시킴
outputs = tf.contrib.layers.fully_connected(inputs = X_for_fc, num_outputs = num_classes, 
                                            activation_fn = None) 


outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits = outputs, targets= Y, weights = weights )# weight를 통해시퀀스의 가중치를 정해주기


loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
prediction = tf.argmax(outputs,axis=2)

TypeError: Tensors in list passed to 'values' of 'ConcatV2' Op have types [int32, float32] that don't all match.

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for step in range(4000):
    l,_ = sess.run([loss,train], feed_dict={X:x_one_hot, Y:y_data})
    result = sess.run(prediction, feed_dict={X:x_one_hot})
    if step % 100 == 0:
        print ("Step:",step,"Loss:",l,"Prediction:", result)
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print ("Prediction:","".join(result_str))
